In [6]:
import matplotlib.pyplot as plt
import sys
import scipy as sp
import numpy as np
import time
import os
from xpcs_viewer import XpcsFile as xf
import scipy.optimize as op
import h5py

In [19]:
def IntegralMat2(Array):
    length = len(Array)
    cs = np.cumsum(Array)
    mat1 = np.tile(cs, length).reshape(length, length)
    deltaMat = np.abs(mat1 - mat1.T)
    return deltaMat

def read_parameters(fname):
    if fname.endswith('.txt'):
        paras = np.loadtxt(fname)
    elif fname.endswith('.csv'):
        paras = np.loadtxt(fname, delimiter=',')
    else:
        paras = None
    return paras


def read_data():
    # save_folder = '/home/8ididata/2022-3/hongrui202210/NonequilibratedAnalysis'
    DataFolderPath = '../data_Twotime'
    DataFolderName = 'C020_Cys0_Exp2_Creep_XPCS_20Pa_01_att02_Lq2_001_0001-8000_Twotime.hdf'
    # Object = xf(os.path.join(DataFolderPath, DataFolderName), cwd='')
    
    phi_list = np.loadtxt('../phi_list.txt')
    phi_length = len(phi_list)
    CreepStart = 1001
    CreepEnd = 2000

    t_length = CreepEnd - CreepStart

    cache_fname = f'c2_cache_{CreepStart}_{CreepEnd}.npz'
    print(cache_fname)
    if os.path.isfile(cache_fname):
        c2_exp = np.load(cache_fname)['c2_exp'].astype(DTYPE)
    else:
        c2_exp = np.zeros((phi_length, t_length, t_length), dtype=DTYPE)
        for phiIdx, phi in enumerate(phi_list):
            cMat = Object.get_twotime_c2(
                    'exchange', phiIdx+1)[CreepStart:CreepEnd, CreepStart:CreepEnd]
            c2_exp[phiIdx] = cMat
        np.savez(cache_fname, c2_exp=c2_exp)
    
    # fix the diagonal line
    c2_exp = fix_diagonal_c2(c2_exp)
    print(c2_exp.shape, c2_exp.dtype)
    return c2_exp.astype(DTYPE), t_length, phi_list.astype(DTYPE), phi_length

def fix_diagonal_c2(c2_3d):
    c2_all = []
    for n in range(c2_3d.shape[0]):
        c2 = c2_3d[n]
        size = c2.shape[0]
        side_band = c2[(np.arange(size - 1), np.arange(1, size))]
        diag_val = np.zeros(size)
        diag_val[:-1] += side_band
        diag_val[1:] += side_band
        norm = np.ones(size)
        norm[1:-1] = 2
        c2[np.diag_indices(c2.shape[0])] = diag_val / norm
    
    return c2_3d

def fit_c2_polynomial(num_v=3, num_d=3, paras_fname=None):
    # read raw data
    c2_exp, t_length, phi_list, phi_length = read_data()
    x0 = np.array([
              1324.1  ,   -0.014, -124.865*0.0054,
              0.003, -0.909,  0.,
              0,
              ])
    # 0.0006395236354457675, -0.4278605424136395, -6.566316071911294e-05,
    for method in (
                'Nelder-Mead',
                # 'Powell',
                # 'L-BFGS-B',
                # 'TNC', 'COBYLA', 'SLSQP', 'trust-constr', 'dogleg',
                # 'trust-ncg', 'trust-exact', 'trust-krylov'):
                ):
    # def chiFunc_polynomial(paras, phi_list, num_d, num_v, c2_exp, method):
        FitOutput = op.minimize(fun=chiFunc_polynomial, x0=x0, 
                                args=(phi_list, num_d, num_v, c2_exp, method),
                                 options={"maxiter": 1000000}, tol=0.00001,
                                method=method)

    FitParas = FitOutput['x']
    
    c2_fit = Homodyne1(FitParas, phi_list)
    shape = c2_fit.shape
    sol_list = []
    for n in range(shape[0]):
        sumy = np.sum(c2_fit[n])
        a = np.array([
            [np.sum(c2_fit[n] ** 2), sumy],
            [sumy, c2_fit[n].size]
        ])
        b = np.array([np.sum(c2_exp[n] * c2_fit[n]), np.sum(c2_exp[n])])
        sol = np.linalg.solve(a, b)
        sol_list.append(sol)

    
    FitParas = np.hstack([FitParas, np.array(sol_list).T.reshape(-1)])
    np.savetxt('../FitParas/C020_FitParas.txt', FitParas, delimiter=",")

def chiFunc_polynomial(paras, phi_list, num_d, num_v, c2_exp, method):
    shape = c2_exp.shape
    residual = 0.0
    t_length = 999
    num_d = 3
    num_v = 3
    
    D_c  = paras[0:num_d]
    v_c  = paras[num_d: num_d + num_v]

    tabs = np.linspace(0.1, 99.9, t_length, dtype=DTYPE) 
    v = v_c[0]*(tabs)**v_c[1] + v_c[2]
    D = D_c[0]*(tabs)**D_c[1] + D_c[2]


    if all(i >= 0 for i in v) == True\
    and all(i >= 0 for i in D)== True\
    :
        c2_fit = Homodyne1(paras, phi_list)
        sol_list = []
        for n in range(shape[0]):
            sumy = np.sum(c2_fit[n])
            a = np.array([
                [np.sum(c2_fit[n] ** 2), sumy],
                [sumy, c2_fit[n].size]
            ])
            b = np.array([np.sum(c2_exp[n] * c2_fit[n]), np.sum(c2_exp[n])])
            sol = np.linalg.solve(a, b)
            residual += np.sum(np.square(c2_fit[n] * sol[0] + sol[1] - c2_exp[n]))
            sol_list.append(sol)

        global gindex
        gindex += 1
        print(f'{gindex:06d}, {method=}, {residual=}')

        if gindex % 500 == 0:
            save_name = f'curr_paras_poly_{gindex:06d}.txt'
            sol_list = np.array(sol_list).T.reshape(-1)
            paras = np.hstack([paras, sol_list])
            np.savetxt(save_name, paras)
            np.savetxt('latest_paras_poly.txt', paras)
        # plt.imshow(c2_fit[0])
        # # plt.imshow(c2_exp[0])
        # plt.show()
    else:
        residual = np.inf
    
    
    return residual



def Homodyne1(x, phi_list):
    phi_length = len(phi_list) 

    x = x.astype(DTYPE)
    t_length = 999
    num_d = 3
    num_v = 3
    
    D_c  = x[0:num_d]
    v_c  = x[num_d: num_d + num_v]
    phio = x[num_d + num_v]

    tabs = np.linspace(0.1, 99.9, t_length, dtype=DTYPE) 
    v = v_c[0]*(tabs)**v_c[1] + v_c[2]
    D = D_c[0]*(tabs)**D_c[1] + D_c[2]
    DMat = IntegralMat2(D)
    vMat = IntegralMat2(v)

    dt = 0.1
    qValue = 0.0054
    gap = 200 * 10000
    
    g1_s = np.exp(qValue**2 / 2 * -abs(DMat) * dt).astype(DTYPE)
    g2_all = np.zeros((phi_length, t_length, t_length), dtype=DTYPE)
    
    for n in range(phi_length):
        w = np.sinc(1/2 /np.pi * qValue* gap * vMat * dt * np.cos(np.deg2rad(phio - phi_list[n])))
        g2_all[n] = (w * g1_s) ** 2

    return g2_all

def Homodyne2(x, phi_list):
    phi_length = len(phi_list) 
    x = x.astype(DTYPE)
    t_length = 999
    num_d = 3
    num_v = 3
    
    D_c  = x[0:num_d]
    v_c  = x[num_d: num_d + num_v]
    D, v, phio, beta, c = separate_params(x, phi_length)
    DMat = IntegralMat2(D)
    vMat = IntegralMat2(v)
    
    dt = 0.1
    qValue = 0.0054
    gap = 200 * 10000
    
    g1_s = np.exp(qValue**2 / 2 * -abs(DMat) * dt).astype(DTYPE)
    g2_all = np.zeros((phi_length, t_length, t_length), dtype=DTYPE)

    for n in range(phi_length):
        w = np.sinc(1/2 /np.pi * qValue* gap * vMat * dt * np.cos(np.deg2rad(phio - phi_list[n])))
        g2_all[n] = c[n] + beta[n] * (w * g1_s ) ** 2
    return g2_all


def separate_params(x, phi_length):
    x = x.astype(DTYPE)
    t_length = 999
    num_d = 3
    num_v = 3
    
    D_c  = x[0:num_d]
    v_c  = x[num_d: num_d + num_v]
    phio = x[num_d + num_v]

    beta = x[num_d + num_v + 1: num_d + num_v + phi_length + 1]
    c = x[num_d + num_v + phi_length + 1: num_d + num_v + 2 * phi_length + 1]
    
    tabs = np.linspace(0.1, 99.9, t_length, dtype=DTYPE) 
    v = v_c[0]*(tabs)**v_c[1] + v_c[2]
    D = D_c[0]*(tabs)**D_c[1] + D_c[2]

    return D, v, phio, beta, c



In [20]:
DTYPE = np.float64
gindex = 0
fit_c2_polynomial()

c2_cache_1001_2000.npz
(23, 999, 999) float64
000001, method='Nelder-Mead', residual=87624.28057112453
000002, method='Nelder-Mead', residual=87448.52672207648
000003, method='Nelder-Mead', residual=87633.08951395183
000004, method='Nelder-Mead', residual=87624.37728726915
000005, method='Nelder-Mead', residual=87110.30577575357
000006, method='Nelder-Mead', residual=90560.49957884877
000007, method='Nelder-Mead', residual=111370.63465394074
000008, method='Nelder-Mead', residual=87624.2782544856
000009, method='Nelder-Mead', residual=96655.19231807394
000010, method='Nelder-Mead', residual=88120.8221731197
000011, method='Nelder-Mead', residual=85827.8716991054
000012, method='Nelder-Mead', residual=88150.9996795036
000013, method='Nelder-Mead', residual=85721.42074114872
000014, method='Nelder-Mead', residual=85564.62754603589
000015, method='Nelder-Mead', residual=85629.81422190025
000016, method='Nelder-Mead', residual=85538.3106148632
000017, method='Nelder-Mead', residual=86061.9